# utils.py

In [248]:
#pip install openpyxl
import os
_path = "../data"
print(os.path.join(_path,"schema.xlsx"))

../data/schema.xlsx


In [255]:
import pandas as pd
import os

_path = "data" #Removed ../ from front of string
_schema = pd.read_excel(os.path.join(_path, "schema.xlsx"), 1, engine='openpyxl')
_schema.loc[:,'VARIABLE_NAME'] = _schema.loc[:,'VARIABLE_NAME'].str.lower()

# Matches the file name with the table name in the schema
#	{filename : table_name}
_table_reference = pd.Series(_schema.loc[:,'table'].values, index=_schema.loc[:,'.csv']).to_dict()
_df_name_reference = pd.Series(_schema.loc[:,'space'].values, index=_schema.loc[:,'.csv']).to_dict() #WIP

# Matches variable names to their translations for each table, respectively (as tables can share variable names but have different meanings)
# 	{table_name: {variable_name : translation}}
_translation_reference = _schema.groupby('TABLE')[['VARIABLE_NAME','TRANSLATION']].apply(lambda g: dict(map(tuple, g.values.tolist()))).to_dict() # Matches 

# Read all CSV's in the directory denoted by <path>
#	{filename : pandas.DataFrame}
dfs = dict([(_df_name_reference.get(csv), pd.read_csv( os.path.join(_path, csv), encoding = 'utf-8' ).rename(columns = _translation_reference.get(_table_reference.get(csv)))) for csv in os.listdir(_path) if csv.endswith('.csv')])


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xde in position 24042: invalid continuation byte

# data_cleaning.py

In [250]:
import pandas as pd
import matplotlib as plt
import numpy as np
from utils import dfs

for key,value in dfs.items():
    print(key)
    
#Check for NaN's    
for key, df in dfs.items():
    nas = df.isna().sum().replace(0,np.nan).dropna() / df.shape[0] * 100
    print(key, '\n\t', nas)

#Transactions
#Replace low NA % categorical with 'unknown'
dfs.get('transaction')['trans_spending_category'].fillna('unknown',inplace=True)
dfs.get('transaction')['transaction_type'].fillna('unknown',inplace=True)
#Dropping high NA % columns
dfs.get('transaction').drop(['merch_x_coord'], axis=1, inplace=True) 
dfs.get('transaction').drop(['merch_y_coord'], axis=1, inplace=True)

#Customer Demographics
#I'm not sure why fillna doens't work on this one, but this works.
median_val = np.nanmedian(dfs.get('customer_demog')['customer_income_level'])
dfs.get('customer_demog')['customer_income_level'] = dfs.get('customer_demog')['customer_income_level'].replace(np.nan,median_val)
#dfs.get('customer_demog')['customer_income_level'].fillna(median_val, inplace=True) #This doesn't work.

#dfs['customer_demog']['customer_income_level'].hist(bins=50,figsize=(20,15))
#save_fig("attribute_histogram_plots")
#plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '../data/schema.xlsx'

# EDA

In [254]:
df_atm = dfs.get('atm_data')
df_atm

In [ ]:
#transfer
#statement
#transaction_statement
#transaction
#customer_demog
#branch_visit
df_test = dfs.get('branch_visit')
df_test.columns

In [ ]:
df_test.head()

In [ ]:
df_test.info()

In [ ]:
#df_test.statement_currency.value_counts()
#df_test.customer_id.isin(df_test.destination_id).value_counts()
#df_test[df_test.customer_id==13261710]
#df_visit_date = pd.to_datetime(df_test['branch_visit_date'], format='%d%b%Y:%H:%M:%S') #Dates
#df_visit_time = pd.to_datetime(df_test['branch_visit_time'], format='%d%b%Y:%H:%M:%S.000000')
#df_visit_time.head()
#df_visit_time.rolling(window = 7).mean()

# Feature Engineering

### Transfers

In [ ]:
#Not currently used
df_transfer = dfs.get('transfer')

### Statements

In [ ]:
df_statement = dfs.get('statement')

#This would be used for getting days from end of month
#df_statement['end_of_month_date'] = pd.to_datetime(df_test['end_of_month_date'], format='%d%b%Y:%H:%M:%S') #Dates
#df_statement['statement_date'] = pd.to_datetime(df_test['statement_date'], format='%d%b%Y:%H:%M:%S') #Dates
#df_statement['statement_day_dif'] = df_statement.end_of_month_date - df_statement.statement_date 
#df_statement = df_statement.drop(['cc_id','statement_date','statement_due_date','statement_amount_USD','statement_amount_Euro'],axis=1)
#df_statement['end_of_month_date'] = df_statement['end_of_month_date'].astype(str)
#df_statement.head()
#Monthly statement amount
#Monthly statement date similarity
#####

df_statement = df_statement.groupby('customer_id')['statement_amount_TL'].mean().to_frame().merge(
    df_statement.groupby('customer_id')['statement_amount_TL'].std().to_frame()
                                                                        , on='customer_id', how='left')
df_statement.columns = ['statement_amount_TL_mean','statement_amount_TL_std']
df_statement

### Payments

In [ ]:
df_payment = dfs.get('transaction_statement')
#Drop USD/Euro.
df_payment=df_payment[df_payment.statement_currency=='TURK_LIRASI']
#Get mean/std of payments
df_payment = df_payment.groupby('customer_id')['payment_amount'].mean().to_frame().merge(df_payment.groupby('customer_id')['payment_amount'].std().to_frame(), on='customer_id', how='left')
df_payment.columns = ['payment_amount_TL_mean','payment_amount_TL_std']
#df_payment

In [ ]:
def test(df):
    return cdist(df[['branch_x_coord','branch_y_coord']], np.array([df[['branch_x_mean','branch_y_mean']].iloc[0,:]])).std()
     
test(df_branch)

In [ ]:
branch_mean

### Branch Visits

In [ ]:
df_branch = dfs.get('branch_visit')
df_visit_time = pd.to_datetime(df_test['branch_visit_time'], format='%d%b%Y:%H:%M:%S.000000')
day_of_week = pd.get_dummies(df_visit_time.dt.dayofweek, prefix = 'BV_DoW') #Branch Visit Day of Week
time_of_day = pd.get_dummies(pd.cut(df_visit_time.dt.hour, bins=4)) #Branch Visit Time of Day
time_of_day.columns = ['BV_very_early','BV_early','BV_later','BV_late']
#df_branch['BV_count']=1
df_branch = df_branch.join(day_of_week).join(time_of_day)
df_branch = df_branch.drop(['branch_visit_date','branch_visit_time','branch_id'],axis=1)

df_branch_xy = df_branch.groupby('customer_id')['branch_x_coord'].mean().to_frame().merge(df_branch.groupby('customer_id')['branch_y_coord'].mean().to_frame(), on='customer_id', how='left')
df_branch_xy.columns = ['branch_x_mean','branch_y_mean']
df_branch_xy['mean_point'] = list(zip(df_branch_xy.branch_x_mean,df_branch_xy.branch_y_mean))
df_branch_xy

df_branch = df_branch.merge(df_branch_xy,on='customer_id',how='left')

#Get branch location mean/variance
df_branch['branch_x_mean'] = df_branch.groupby('customer_id')['branch_x_coord'].transform('mean')
df_branch['branch_y_mean'] = df_branch.groupby('customer_id')['branch_y_coord'].transform('mean')
branch_mean = df_branch.groupby('customer_id')[['branch_y_mean','branch_x_mean','branch_x_coord','branch_y_coord']].apply(
    lambda df: cdist(df[['branch_x_coord','branch_y_coord']], np.array([df[['branch_x_mean','branch_y_mean']].iloc[0,:]])).mean()).to_frame()
branch_mean.columns = ['branch_mean_distance']

df_branch = df_branch.merge(branch_mean,on='customer_id',how='left')
df_branch = df_branch.drop(['branch_x_coord','branch_y_coord','branch_x_mean','branch_y_mean','mean_point'],axis=1)

#Group by for sums
df_branch = df_branch.groupby('customer_id')[df_branch.columns[2:]].sum()
#Final clean-up
df_branch

In [ ]:
np.linalg.norm((0,1),(0,2))

### Transactions

In [ ]:
df_transactions = dfs.get('transaction')
#Drop USD/Euro.
#df_transactions=df_transactions[df_transactions.currency=='TURK_LIRASI']
#Transaction Category to columns
tran_cat_list = df_transactions.trans_spending_category.value_counts()[1:9].index.tolist()
df_transactions['trans_spending_category'] = df_transactions['trans_spending_category'].apply(lambda i: i if i in tran_cat_list else 'other')
df_transactions = df_transactions[['customer_id','trans_spending_category','transaction_total']]
df_tran_dummy = pd.get_dummies(df_transactions.trans_spending_category,prefix='tran_spend_cat_')
df_transactions = pd.concat([df_transactions,df_tran_dummy],axis=1).drop('trans_spending_category',axis=1)
for (columnName, columnData) in df_transactions.iteritems():
    if columnName not in ['customer_id','transaction_total']:
        df_transactions[columnName] = df_transactions[columnName]*df_transactions['transaction_total']
df_transactions = df_transactions.drop('transaction_total',axis=1)
df_transactions = df_transactions.groupby('customer_id').sum()
df_transactions

transfer
  Matching customer_id/destination_id
statement
    Similarity of statement date/statement due date?
    Average Statement Amount
    #All in sample are turkish lira
transaction_statement
    Similarity of statement date/statement amount
transaction
    date/time/total/spending category
customer_demog
    Merge Both
branch_visit
    Same date/similar time of visit *Swapped for OHE Day of week/time of day sums
        7 variables for Day of Week
        4 variables for Time of Day (Used even number since many were centered around middle 2 hours of day)
    Same branch id (same as x/y coord) *Dropped ID for x/y

In [ ]:
#This section was for an early model concept using male/female pairs as rows

#Married Only
#df_married = dfs.get('customer_demog')[dfs.get('customer_demog').customer_marital_status=='EVLİ']
#All possible male/female pairs
#df_married_male = df_married[df_married.customer_gender=='E']
#df_married_female = df_married[df_married.customer_gender=='K']
#df_married_male.columns = 'm_'+df_married_male.columns
#df_married_female.columns = 'f_'+df_married_female.columns
#df_potential_pairs = df_married_male.merge(df_married_female, how='cross')

#df_potential_pairs['sameday_branch_visits']=list(zip(df_potential_pairs.b_x,df_potential_pairs.b_y))
#df_potential_pairs['sameday_branch_visits'] = df_potential_pairs['sameday_branch_visits'].map(intersection_counter)

### Customer Demographics

In [ ]:
df_customer_demog = dfs.get('customer_demog')
lst_risk_codes = ['1)RISKSIZ','2)GECIKME 1-15 GUN','4)GECIKME 30-59 GUN','5)GECIKME 60+ GUN','6)TAKIP']

j=0
for i in lst_risk_codes:
    df_risk_code = df_customer_demog[['kk_risk_kodu_201407', 'kk_risk_kodu_201408', 'kk_risk_kodu_201409', 'kk_risk_kodu_201410', 'kk_risk_kodu_201411', 'kk_risk_kodu_201412', 'kk_risk_kodu_201501', 'kk_risk_kodu_201502', 'kk_risk_kodu_201503', 'kk_risk_kodu_201504', 'kk_risk_kodu_201505', 'kk_risk_kodu_201506']]
    df_risk_code = df_risk_code.replace(i,1)
    df_risk_code[df_risk_code!=1] = 0
    df_risk_code = df_risk_code.sum(axis='columns')
    df_customer_demog[lst_risk_codes[j]] = df_risk_code
    j+=1

df_customer_demog = df_customer_demog[['customer_id','branch_id','customer_home_x_coord','customer_home_y_coord','customer_gender','customer_marital_status','customer_education_level','customer_income_level','customer_age','akbank_banking_age','1)RISKSIZ','2)GECIKME 1-15 GUN','4)GECIKME 30-59 GUN','5)GECIKME 60+ GUN','6)TAKIP']]
df_customer_demog['branch_id']=df_customer_demog['branch_id'].astype(str)
#df_customer_demog.info()

# Combine Data Tables

In [247]:
#transfer
#statement
#payment
#transactions
#customer_demog
#branch
df_customer_demog
df_statement
#df_transfer
df_payment
df_transactions
df_branch

df_merged = df_customer_demog.merge(df_branch,on='customer_id',how='left')
df_merged = df_merged.merge(df_statement,on='customer_id',how='left')
df_merged = df_merged.merge(df_payment,on='customer_id',how='left')
df_merged = df_merged.merge(df_transactions,on='customer_id',how='left')

df_merged

,customer_id,branch_id,customer_home_x_coord,customer_home_y_coord,customer_gender,customer_marital_status,customer_education_level,customer_income_level,customer_age,akbank_banking_age,...,payment_amount_TL_std,tran_spend_cat__AKARYAKIT,tran_spend_cat__DİĞER,tran_spend_cat__HİZMET SEKTÖRLERİ,tran_spend_cat__RESTORAN,tran_spend_cat__SAĞLIK,tran_spend_cat__TEKNOLOJİ,tran_spend_cat__TEKSTİL,tran_spend_cat__TELEKOMÜNİKASYON,tran_spend_cat__other
0,25139695,2,36.971366,35.362328,E,EVLİ,LİSE,2500.0,55,2,...,73.897226,550.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1365.15
1,21213035,2,37.010670,35.369540,E,BİLINMİYOR,LİSE,2500.0,24,4,...,269.825782,0.00,200.00,0.00,42.0,0.00,0.00,0.00,1522.50,1405.06
2,21242221,2,40.920454,29.187872,E,BEKAR,ORTAOKUL,1200.0,26,4,...,299.253714,0.00,330.00,4.00,67.0,0.00,0.00,0.00,0.00,312.84
3,23187800,2,41.100502,28.892116,E,EVLİ,LİSE,850.0,39,3,...,396.460906,0.00,100.00,63.00,0.0,0.00,0.00,0.00,0.00,7501.75
4,16400475,2,41.008290,29.083431,E,EVLİ,ÜNİVERSİTE,NaN,45,8,...,5953.488792,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,83443.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20186282,4,41.017983,28.666377,E,BEKAR,LİSE,4500.0,25,5,...,164.083718,0.00,0.00,20.50,0.0,50.00,0.00,100.00,701.20,3323.98
96,20726978,4,40.995440,28.767980,E,BEKAR,YÜKSEKOKUL,1500.0,25,5,...,127.863641,25.25,10.00,192.50,0.0,38.00,75.00,324.43,51.00,1338.67
97,17638957,4,41.025290,28.698740,E,EVLİ,LİSE,4000.0,38,7,...,523.457319,2854.80,532.50,3.00,361.7,179.71,253.17,1751.90,335.00,14989.99
98,16948506,4,40.994817,28.769029,E,EVLİ,LİSE,6000.0,38,8,...,405.465583,420.00,0.00,0.00,93.5,445.00,658.90,695.27,0.00,11863.84
